In [ ]:
import pandas as pd


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
rpi = pd.read_csv("../2023/rpi.csv", index_col=False)

In [ ]:
rpi.columns

In [ ]:
rpi_cols = [
    "RPI",
    "Record",
    "Team"
]

In [ ]:
rpi = rpi.loc[:, rpi_cols]

In [ ]:
rpi.head()

In [ ]:
temp_df = rpi["Team"].str.split(
    pat="  ",
    n=1,
    expand=True
)

In [ ]:
rpi["conf_rec"] = temp_df[1]
rpi["Team"] = temp_df[0]

In [ ]:
rpi = rpi.drop(["conf_rec"], axis=1)

In [ ]:
rpi = rpi[rpi["RPI"] != "RPI"]

In [ ]:
rpi.head(50)

In [ ]:
def make_record_list(record: str) -> list:
    rec_list = record.split("-")
    return rec_list

In [ ]:
rpi["Record"] = rpi["Record"].apply(make_record_list)
rpi.head()

In [ ]:
def get_wins(record: list) -> int:
    return record[0]

def get_losses(record: list) -> int:
    return record[1]

def get_ties(record: list) -> int:
    if len(record) > 2:
        return record[2]
    else:
        return 0

In [ ]:
rpi["wins"] = rpi["Record"].apply(get_wins)
rpi["losses"] = rpi["Record"].apply(get_losses)
rpi["ties"] = rpi["Record"].apply(get_ties)

rpi = rpi.drop(["Record"], axis=1)

rpi.head()

In [ ]:
def get_wins_ties(w, t):
    return float(w) + 0.5 * float(t)

def get_losses_ties(l, t):
    return float(l) + 0.5 * float(t)

In [ ]:
rpi["wins"] = rpi.apply(lambda team: get_wins_ties(team["wins"], team["ties"]), axis=1)
rpi["losses"] = rpi.apply(lambda team: get_losses_ties(team["losses"], team["ties"]), axis=1)

rpi = rpi.drop(["ties"], axis=1)

rpi.head(50)

In [ ]:
rpi.head()

In [ ]:
sos = pd.read_csv("../2023/sos.csv", index_col=False)

In [ ]:
sos.head(50)

In [ ]:
def get_opp_record_list(opp_record: str) -> list:
    return opp_record.split("-")

In [ ]:
sos["Opp Record"] = sos["Opp Record"].apply(get_opp_record_list)

In [ ]:
def get_opp_wins(opp_record: list) -> float:
    return float(opp_record[0])

def get_opp_losses(opp_record: list) -> float:
    return float(opp_record[1])

In [ ]:
sos["opp_wins"] = sos["Opp Record"].apply(get_opp_wins)
sos["opp_losses"] = sos["Opp Record"].apply(get_opp_losses)

sos = sos.drop(["Unnamed: 0", "Opp Record"], axis=1)
sos.head(50)

In [ ]:
def make_string(value):
    return str(value)

In [ ]:
rpi["Team"] = rpi["Team"].apply(make_string)
sos["Team"] = sos["Team"].apply(make_string)

In [ ]:
rpi_sos = rpi.merge(sos, on="Team", how='inner')

rpi_sos.head(50)

In [ ]:
rpi_sos = rpi_sos.sort_values("Team")

rpi_sos.head(50)

rpi_sos.to_csv("2023_rpi_sos.csv")